In [45]:

import numpy as np
import pandas as pd


# Charger
df = pd.read_csv(r"../data/raw/full_2020.csv", low_memory=False)
print("Shape brut:", df.shape)


Shape brut: (3522416, 40)


In [ ]:
# =========================
# 1) CHARGEMENT & NETTOYAGE DE BASE
# - Garde uniquement les VENTES
# - Transforme la date et crée annee/mois
# - Convertit les colonnes numériques clés
# - Supprime les valeurs impossibles
# - Trim "doux" des outliers via prix/m² (1e–99e pct)
# =========================
import numpy as np
import pandas as pd


print("Shape brut:", df.shape)

# Garder uniquement les ventes
if "nature_mutation" in df.columns:
    df = df[df["nature_mutation"].astype(str).str.lower().str.contains("vente", na=False)]

# Dates -> annee, mois
if "date_mutation" in df.columns:
    df["date_mutation"] = pd.to_datetime(df["date_mutation"], errors="coerce")
    df["annee"] = df["date_mutation"].dt.year
    df["mois"]  = df["date_mutation"].dt.month

# Colonnes numériques importantes
for col in ["valeur_fonciere","surface_reelle_bati","surface_terrain",
            "nombre_pieces_principales","longitude","latitude"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# onléve les Valeurs impossibles

if "valeur_fonciere" in df.columns:
    df = df[df["valeur_fonciere"] > 0]
if "surface_reelle_bati" in df.columns:
    df = df[df["surface_reelle_bati"] > 0]
if "surface_terrain" in df.columns:
    df = df[df["surface_terrain"] >= 0]

# (on enlève juste les 1% les plus bas et les 1% les plus hauts) garde 98% des ventes normales
if {"valeur_fonciere","surface_reelle_bati"}.issubset(df.columns):
    prix_m2 = df["valeur_fonciere"] / df["surface_reelle_bati"].replace(0, np.nan)
    q1, q99 = prix_m2.quantile([0.01, 0.99])
    df = df[(prix_m2 >= q1) & (prix_m2 <= q99)]

print("Shape après nettoyage:", df.shape)



Shape brut: (3522416, 40)
Shape après nettoyage: (840758, 42)


In [ ]:
# =========================
# 2) LOCALISATION SIMPLE SANS MODÈLE
# - Normalise code_postal (ex : 1250.0 -> "01250")
# - Crée departement = 2 premiers chiffres du code postal
# - Ajoute commune_freq = nb de ventes par commune
# - Supprime longitude/latitude et geo_cluster
# =========================

# Code postal propre sur 5 caractères
if "code_postal" in df.columns:
    df["code_postal"] = pd.to_numeric(df["code_postal"], errors="coerce").astype("Int64")
    df["code_postal"] = df["code_postal"].astype(str).str.zfill(5)
else:
    df["code_postal"] = np.nan

# Département = 2 premiers chiffres
df["departement"] = df["code_postal"].str[:2]

# Fréquence de ventes par commune
if "nom_commune" in df.columns:
    commune_freq = df["nom_commune"].value_counts()
    df["commune_freq"] = df["nom_commune"].map(commune_freq).astype("float32")
else:
    df["commune_freq"] = np.nan

# Supprime les coordonnées (choix de simplicité)
if {"longitude","latitude"}.issubset(df.columns):
    df = df.drop(columns=["longitude","latitude"])

# Supprime geo_cluster si présent
if "geo_cluster" in df.columns:
    df = df.drop(columns=["geo_cluster"])


In [48]:
# =========================
# 3) FEATURES SUPPLÉMENTAIRES LÉGÈRES
# - log_surface 
# - densite_pieces = pièces / surface
# =========================
#On calcule le nombre de pièces par m².
df["log_surface_reelle_bati"] = np.log1p(df["surface_reelle_bati"])
df["densite_pieces"] = df["nombre_pieces_principales"] / df["surface_reelle_bati"]
df["densite_pieces"] = df["densite_pieces"].replace([np.inf, -np.inf], np.nan)



In [ ]:
# =========================
# 4) SÉLECTION DES FEATURES
# =========================

candidate_features = [
    "log_surface_reelle_bati",
    "densite_pieces",
    "nombre_pieces_principales",
    "surface_terrain",
    "type_local",
    "departement",
    "commune_freq",
    "annee", "mois"
]

feature_cols = [c for c in candidate_features if c in df.columns]
df_model = df.dropna(subset=["valeur_fonciere"]).copy()
X = df_model[feature_cols].copy()
y = df_model["valeur_fonciere"].astype(float).copy()


In [ ]:
# =========================
# 5) PRÉPARATION & ENCODAGE
# - Catégorielles : OneHot (type_local, departement)
# - Numériques : pass-through
# =========================

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


cat_cols = [c for c in ["type_local","departement"] if c in X.columns]
num_cols = [c for c in X.columns if c not in cat_cols]

# Cast
for c in cat_cols:
    X[c] = X[c].astype("string")
for c in num_cols:
    X[c] = pd.to_numeric(X[c], errors="coerce")
#nepastransformerlesnum
preprocess = ColumnTransformer([
    ("num", "passthrough", num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
])


In [ ]:
import time
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np, time
from sklearn.model_selection import train_test_split
# =========================
# 6) ENTRAÎNEMENT RANDOM FOREST
# =========================

max_rows = 120_000
if len(X) > max_rows:
    idx = np.random.RandomState(42).choice(X.index, size=max_rows, replace=False)
    X = X.loc[idx].copy()
    y = y.loc[idx].copy()
print("Taille entraînement après downsample :", len(X))


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=80,       
    max_depth=18,         
    max_features="sqrt",  
    bootstrap=True,
    max_samples=0.65,     
    n_jobs=-1,
    random_state=42
)

model = Pipeline([("prep", preprocess), ("rf", rf)])


t0 = time.time()
model.fit(X_train, np.log1p(y_train))
print("Fit terminé en", round(time.time()-t0,1), "s")

# Prédictions & métriques
y_pred = np.expm1(model.predict(X_test))
mae  = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2   = r2_score(y_test, y_pred)


print("MAE :", round(mae,2), "| RMSE :", round(rmse,2), "| R² :", round(r2,4))


Taille entraînement après downsample : 120000
Fit terminé en 10.3 s
MAE : 393608.92 | RMSE : 4337603.09 | R² : 0.1868


In [ ]:
#Augmenter la taille du downsample → 200k ou 300k lignes .

#Augmenter (n_estimators=200) et augmenter max_depth (20–25).

#ajouter plus de features